## Recursive inference (multi-step) for BSCTRFM models

### use time series instead of SLDB arrays for easier and more efficient timestamp management

In [1]:
import os
import json
import numpy as np
import pandas as pd
import joblib
from datetime import datetime
from math import sqrt
from sklearn.metrics import mean_squared_error, mean_absolute_error

# uncomment the following line for compatibility with TensorFlow 1.15 (on GCP)
# import tensorflow.compat.v1 as tf
# uncomment the following line for TensorFlow 2.X (local execution)
import tensorflow as tf

# forecast model was saved in TensorFlow 1.15
# but, in order to make predictions locally, has to be loaded with TensorFlow 2
from tensorflow.saved_model import load

In [2]:
# symmetric mean absolute percentage error
def symmetric_mean_absolute_percentage_error(targets, predictions):
    '''
    predictions: a list with the predicted values
    targets: a list with the actual values
    '''
    import numpy as np
    # lists to NumPy arrays
    targets, predictions = np.array(targets), np.array(predictions)
    # verify predictions and targets have the same shape
    if predictions.shape == targets.shape:
            return(np.sum(2*np.abs(predictions - targets) /
                          (np.abs(targets) + np.abs(predictions)))/predictions.shape[0])

In [3]:
# a function to encode float values for serialized examples
def _float_feature_from_list_of_values(list_of_values):
    """Returns a float_list from a list of floats / doubles."""
    return tf.train.Feature(float_list=tf.train.FloatList(value=list_of_values))

In [4]:
# converts a set of tensors to a feature dict to a serialized example to pass it
# to the prediction function of the saved model 
def input_tensors_to_serialized_example(encoder_input_float_tensor,
                                        decoder_input_float_tensor,
                                        id_float_tensor):
    # first, pass the float tensors to NumPy array, then flatten them
    encoder_input_flat_array = encoder_input_float_tensor.numpy().flatten()
    decoder_input_flat_array = decoder_input_float_tensor.numpy().flatten()
    id_flat_array = id_float_tensor.numpy().flatten()
    
    # second, build the protobuffer example
    example = tf.train.Example(
        # features within the example
        features=tf.train.Features(
            # feature definition
            feature={
                'encoder_input': _float_feature_from_list_of_values(encoder_input_flat_array),
                'decoder_input': _float_feature_from_list_of_values(decoder_input_flat_array),
                'id': _float_feature_from_list_of_values(id_flat_array)
            }
        )
    )    
    # third, serialize the example dictionary to a string
    serialized_example = example.SerializeToString()
    # fourth, wrap the serialized example as a NumPy-string array
    numpy_example = np.array(serialized_example, dtype='S')
    # fifth, wrap the NumPy-string array as a string tensor
    serialized_example = tf.convert_to_tensor(numpy_example)

    return serialized_example

In [5]:
PROJECT_ROOT = '/home/developer/gcp/cbidmltsf'

In [6]:
# during batch prediction, the SLDB identifier is obtained via Abseil Flags
# THE SLDB FOR INFERENCE MUST BE THE SAME USED FOR TRAINING! (THE ONE SETUP IN THE CONFIGURATION FILE)
sldb_id = 'LD2011-2014_SEPARATED_FULL_BSCTRFM_168_168_07DB_MMX'

In [7]:
# during batch prediction, the dataset name is obtained via Abseil Flags
dataset = 'test'

In [8]:
# define a forecast window to guide the iterative prediction process
# start with a hourly, day-ahead process
forecast_window = 24

In [9]:
# ADD AN INFERENCE IDENTIFIER, BECAUSE FOR TRANSFORMER-BASED MODELS, DIFFERENT INFERENCES
# CAN BE PRODUCED FROM A SINGLE SAVED MODEL (USUALLY TO PRODUCE DIFFERENT FORECAST WINDOWS)
# during batch prediction, the inference identifier should be obtained via Abseil Flags
inference = '{:03d}'.format(forecast_window)

In [10]:
# build a path to the SLDB json file
data_dir = '{}/{}/{}'.format(PROJECT_ROOT, 'sldbs', sldb_id)

# then get the ts_identifier from the json file in the sldb directory
sldb_json_file = '{}/sldb.json'.format(data_dir)

In [11]:
# open the json file
with open(sldb_json_file, 'r') as inputfile:
    sldb_dict = json.load(inputfile)

In [12]:
# and get the time series identifier
ts_identifier = sldb_dict['ts']
ts_identifier

'LD2011-2014_SEPARATED_FULL'

In [15]:
m = 168
m

168

In [16]:
t = 168
t

168

In [23]:
model_ids = [
    # BSCTRFM_TPU_055_00, at 3K, 6K, 9K, 12K train_steps
    'BSCTRFM_TPU_055', 'BSCTRFM_TPU_055', 'BSCTRFM_TPU_055', 'BSCTRFM_TPU_055',
    # BSCTRFM_TPU_055_01, at 3K, 6K, 9K, 12K train_steps
    'BSCTRFM_TPU_055', 'BSCTRFM_TPU_055', 'BSCTRFM_TPU_055', 'BSCTRFM_TPU_055',
    # BSCTRFM_TPU_055_02, at 6K, 9K, 12K train_steps
    'BSCTRFM_TPU_055', 'BSCTRFM_TPU_055', 'BSCTRFM_TPU_055',
    # BSCTRFM_TPU_055_03, at 9K, 12K train_steps
    'BSCTRFM_TPU_055', 'BSCTRFM_TPU_055',
    # BSCTRFM_TPU_056_00, at 4K, 5K, 6K, 7K, 8K train_steps
    'BSCTRFM_TPU_056', 'BSCTRFM_TPU_056', 'BSCTRFM_TPU_056', 'BSCTRFM_TPU_056', 'BSCTRFM_TPU_056',
    # BSCTRFM_TPU_056_01, at 4K, 6K, 7K, 8K train_steps
    'BSCTRFM_TPU_056', 'BSCTRFM_TPU_056', 'BSCTRFM_TPU_056', 'BSCTRFM_TPU_056',
]

In [24]:
executions = [
    # BSCTRFM_TPU_055_00, at 3K, 6K, 9K, 12K train_steps
    0, 0, 0, 0,
    # BSCTRFM_TPU_055_01, at 3K, 6K, 9K, 12K train_steps
    1, 1, 1, 1,
    # BSCTRFM_TPU_055_02, at 6K, 9K, 12K train_steps
    2, 2, 2,
    # BSCTRFM_TPU_055_03, at 9K, 12K train_steps
    3, 3,
    # BSCTRFM_TPU_056_00, at 4K, 5K, 6K, 7K, 8K train_steps
    0, 0, 0, 0, 0,
    # BSCTRFM_TPU_056_01, at 4K, 6K, 7K, 8K train_steps
    1, 1, 1, 1,
]

In [25]:
saved_model_ids = [
    # BSCTRFM_TPU_055_00, at 3K, 6K, 9K, 12K train_steps
    '1637863002', '1637863629', '1637864194', '1637865732',
    # BSCTRFM_TPU_055_01, at 3K, 6K, 9K, 12K train_steps
    '1637863302', '1637863905', '1637864474', '1637865979',
    # BSCTRFM_TPU_055_02, at 6K, 9K, 12K train_steps
    '1637864792', '1637865102', '1637866231',
    # BSCTRFM_TPU_055_03, at 9K, 12K train_steps
    '1637865427', '1637866499',
    # BSCTRFM_TPU_056_00, at 4K, 5K, 6K, 7K, 8K train_steps
    '1637866890', '1637867407', '1637867659', '1637867918', '1637868158',
    # BSCTRFM_TPU_056_01, at 4K, 6K, 7K, 8K train_steps
    '1637867159', '1637868463', '1637868718', '1637868965',
]

In [46]:
for model_id, execution, saved_model_id in zip(model_ids,
                                               executions,
                                               saved_model_ids):
    print(model_id, execution, saved_model_id)

BSCTRFM_TPU_055 0 1637863002
BSCTRFM_TPU_055 0 1637863629
BSCTRFM_TPU_055 0 1637864194
BSCTRFM_TPU_055 0 1637865732
BSCTRFM_TPU_055 1 1637863302
BSCTRFM_TPU_055 1 1637863905
BSCTRFM_TPU_055 1 1637864474
BSCTRFM_TPU_055 1 1637865979
BSCTRFM_TPU_055 2 1637864792
BSCTRFM_TPU_055 2 1637865102
BSCTRFM_TPU_055 2 1637866231
BSCTRFM_TPU_055 3 1637865427
BSCTRFM_TPU_055 3 1637866499
BSCTRFM_TPU_056 0 1637866890
BSCTRFM_TPU_056 0 1637867407
BSCTRFM_TPU_056 0 1637867659
BSCTRFM_TPU_056 0 1637867918
BSCTRFM_TPU_056 0 1637868158
BSCTRFM_TPU_056 1 1637867159
BSCTRFM_TPU_056 1 1637868463
BSCTRFM_TPU_056 1 1637868718
BSCTRFM_TPU_056 1 1637868965


In [47]:
encoder_input_columns = [
    'kw_scaled',
    'sin_hours_from_start',
    'cos_hours_from_start',
    'sin_hour_day',
    'cos_hour_day',
    'sin_day_week',
    'cos_day_week',
    # 'sin_day_month',
    # 'cos_day_month',
    # 'sin_day_year',
    # 'cos_day_year'
]

In [48]:
decoder_input_columns = encoder_input_columns

In [49]:
id_columns = ['token_id']

In [52]:
# use a dictionary to remain the code consistent with the SLDB building process
# most of the times, only ts['test'] will be used for inference
# however, ts['eval'] might also be used, as it have not really been seen by training process
# (no modification in model training resulted from evaluation stage)

### rename the time series dictionary to ts_test, use the customer_id as key

In [53]:
start, end = 1, 370

In [54]:
customer_ids = ['MT_{:03d}'.format(token_id) for token_id in np.arange(start, end + 1)]

In [55]:
len(customer_ids)

370

## skip the following code to avoid inference process

In [56]:
# a second inference identifier to run more than one inference
# on the same combination model_id, execution, inference;
# to produce different inference processes because predict_fn is stochastic
event = 0

for model_id, execution, saved_model_id in zip(model_ids,
                                               executions,
                                               saved_model_ids):

    # use model identifier and execution number to build the model directory string
    model_dir = '{}_{:02d}'.format(model_id, execution)

    # get the path to the saved model main directory
    saved_model_path = '{}/{}/{}/export/exporter'.format(PROJECT_ROOT,
                                                         'models',
                                                         model_dir)

    # get all the files in the saved model path, to find the most recent one
    # all_files = os.listdir(saved_model_path)
    # get the path to the most recent saved model
    # latest_saved_model_id = sorted(all_files)[-1]

    # build the full path for the latest saved model dir
    export_dir = '{}/{}'.format(saved_model_path, saved_model_id)
    print ('Exported model path is {}'.format(export_dir))

    # load the saved model and the prediction function
    imported = load(export_dir=export_dir, tags='serve')
    predict_fn = imported.signatures["serving_default"]
    

    ts_test = dict()

    # a dictionary to store predictions detail dataframe per station id
    predictions_detail = dict()
    
    for customer_id in customer_ids:

        # read the time series for the current customer
        ts_test[customer_id] = pd.read_pickle('{}/test/{}.pkl'.format(data_dir, customer_id))

        # for consistency, rename the column 'date' as 'timestamp'
        ts_test[customer_id] = ts_test[customer_id].rename(columns={"date": "timestamp"})
        # set the column 'timestamp' as index
        ts_test[customer_id] = ts_test[customer_id].set_index('timestamp')

        # report start and end timestamp for the loaded time series
        print('Loaded test time series for {}, which spans from {} to {}'.\
             format(customer_id,
                    str(ts_test[customer_id].index[:1][0]),
                    str(ts_test[customer_id].index[-1:][0])))

        scaler_type = 'min_max'

        if scaler_type == 'min_max':
            # build a path to the scaler of the time series
            scaler_path = '{}/scalers/min_max_{}.save'.format(data_dir, customer_id)
            # and load it
            scaler = joblib.load(scaler_path)


        '''
        if scaler_type == 'standard':
            # build a path to the scaler of the time series
            scaler_path = '{}/scalers/standard_{}.save'.format(data_dir, customer_id)
            # and load it
            scaler = joblib.load(scaler_path)
        '''    


        # a columns list for the predictions dataframe
        pred_df_columns = ['model_id',
                           'execution',
                           'dataset',
                           'inference',
                           'customer_id',
                           'string_timestamps',
                           'predictions',
                           'targets']

        # build the predictions dataframe as a key-value pair of the dictionary
        predictions_detail[customer_id] = pd.DataFrame(columns=pred_df_columns)

        # iterate on a set of valid rows of the test dataset
        # starting_point = 0 # based on the inference dataset
        # span = 1 + 6*24 # number of days in the test dataset, expressed in hours

        # MAKE THE PREDICTION SETS FOR BENCHMARKING ONLY! 
        # dataset_row_indexes_list = starting_point + np.arange(span)
        dataset_row_indexes_list = [0, 24, 48, 72, 96, 120, 144]

        for start_index in dataset_row_indexes_list:

            # define first prediction interval with start- and end-index
            # given the interval time_series[start_index:end_index]
            # the conditioning range is the union of the encoder-input and the decoder-input
            # and the prediction range is only the last lecture in the interval,
            # by means of a recursive inference process
            # on each step the last prediction is added to the decoder input
            # and the prediction range grows one step into the future

            # get the end-index of this recursive inference interval
            end_index = start_index + (m + t)

            # initialize a list to store recurrent predictions for this interval
            predictions_list = list()

            for i in np.arange(forecast_window):

                # build the inference interval as a sub-series of the dataset
                # the following slicing operation is equivalent to:
                # sub_series = ts_test[customer_id].iloc[start_index + i : end_index + i]
                sub_series = ts_test[customer_id][start_index + i : end_index + i]

                # important: build sources as copies of the sub-series (and therefore of the global time series)
                # to avoid overwriting the original dataset

                # the encoder input source
                encoder_input = sub_series[encoder_input_columns][:m].copy()

                # the decoder input source
                # decoder_input = sub_series[m-1:-1].copy()
                decoder_input = sub_series[decoder_input_columns][m-1:m-1+t].copy()

                # the id (integer) for the customer
                id_input = sub_series[id_columns][:1].copy()

                # on first step (i=0), the decoder input carries only true values
                # and the predictions list is empty
                # on subsequent steps, the decoder input includes all previous predictions
                # (stored in the predictions list)
                if i > 0:
                    decoder_input['kw_scaled'][-i:] = predictions_list

                # the target source, for metrics calculation
                # the first part of the sub-series is the encoder input, and
                # the second part of the sub-series is the target (only the variable column!)
                target = sub_series['kw_scaled'][m:].copy()

                # build source tensors from the sub-series    
                encoder_input_tensor = tf.expand_dims(encoder_input, axis=0)
                decoder_input_tensor = tf.expand_dims(decoder_input, axis=0)
                id_tensor = tf.expand_dims(id_input, axis=0)

                # make input example for the prediction function
                input_example = input_tensors_to_serialized_example(encoder_input_tensor,
                                                                    decoder_input_tensor,
                                                                    id_tensor)

                # get the output of the prediction function as a dictionary
                predict_output_dict = predict_fn(input_example)

                # get the prediction output tensor
                predict_output_tensor = predict_output_dict['forecast']

                # get the most recent prediction
                most_recent_prediction = predict_output_tensor[0, :, 0].numpy()[-1]

                # append the most recent prediction timestep to the predictions list
                predictions_list.append(most_recent_prediction)

                # pass the predictions list to an array
                # current_predictions_array = np.array(predictions_list).reshape(-1, 1)
                # get the targets vector to be compared with the current predictions array
                # current_targets = np.array(target[-i-1:]).reshape(-1, 1)

                # calculate SMAPE on the rescaled variable
                # rescaled_predictions = min_max_scaler.inverse_transform(current_predictions_array)
                # rescaled_targets = min_max_scaler.inverse_transform(current_targets)

                # current_smape = symmetric_mean_absolute_percentage_error(
                #     rescaled_targets, rescaled_predictions)
                # print('On row {}, SMAPE for the first {} rescaled prediction(s) is {}'.format(start_index,
                #                                                                               i + 1,
                #                                                                               current_smape))        

            # iterative predictions over the forecast window reside in predictions_list
            # convert list to array, then expand feature dimension with value 1
            predicted_values = np.array(predictions_list).reshape(-1, 1)

            # inverse-scale predictions
            rescaled_predicted_values = scaler.inverse_transform(predicted_values)

            # and the true values remain in the prediction tensor, pass them to a NumPy array
            # for the true values array, expand feature dimension with value 1
            true_values = np.array(target[-i-1:]).reshape(-1, 1)

            # inverse-scale true values
            rescaled_true_values = scaler.inverse_transform(true_values)

            # a temporary dataframe built from the data in the current row
            df = pd.DataFrame(columns=pred_df_columns)
            df['model_id'] = [model_id]
            df['execution'] = [execution]
            df['dataset'] = [dataset]
            df['inference'] = [inference]
            df['customer_id'] = [customer_id]
            df['string_timestamps']= [pd.to_datetime(target.index[-i-1:]).astype(str).tolist()]
            df['predictions'] = [np.squeeze(rescaled_predicted_values).tolist()]
            df['targets'] = [np.squeeze(rescaled_true_values).tolist()]

            # calculate mean absolute error and normalized deviation
            mae = mean_absolute_error(rescaled_true_values, rescaled_predicted_values)
            df['mae'] = mae

            true_values_average = np.mean(rescaled_true_values)
            df['nd'] = mae/true_values_average

            # calculate root mean squared error and normalized root mean squared error
            rmse = sqrt(mean_squared_error(rescaled_true_values, rescaled_predicted_values))
            df['rmse'] = rmse
            df['nrmse'] = rmse/true_values_average

            df['smape'] = symmetric_mean_absolute_percentage_error(rescaled_true_values,
                                                                   rescaled_predicted_values)

            # append the temporary dataframe to the predictions detail dataframe
            predictions_detail[customer_id] = pd.concat([predictions_detail[customer_id], df])


        # reset the index of final dataframe, once all of its rows (dataset) have been processed
        predictions_detail[customer_id] = predictions_detail[customer_id].reset_index(drop=True)


        # hard-wired switcher to persist dataframe
        persist_detail = True

        if persist_detail:
            # build a path to persist the dataframe to database/predictions_detail/
            detail_pickle_path = '{}/{}/{}/{}_{:02d}_{}_{}_{}_{:02d}.pkl'.format(
                PROJECT_ROOT,
                'database',
                'predictions_detail',
                model_id,
                execution,
                saved_model_id,
                # for electricity dataset, replace dataset with customer_id
                customer_id,
                inference,
                event)

            # persist the Pandas dataframe to database/predictions_detail/
            predictions_detail[customer_id].to_pickle(detail_pickle_path)
            print(
                'Persisted predictions detail of {}_{:02d}_{}_{}_{}_{:02d}'.format(model_id,
                                                                                   execution,
                                                                                   saved_model_id,
                                                                                   customer_id,
                                                                                   inference,
                                                                                   event)
            )

Exported model path is /home/developer/gcp/cbidmltsf/models/BSCTRFM_TPU_055_00/export/exporter/1637863002
Loaded test time series for MT_001, which spans from 2014-08-18 01:00:00 to 2014-09-07 23:00:00
Persisted predictions detail of BSCTRFM_TPU_055_00_1637863002_MT_001_024_00
Loaded test time series for MT_002, which spans from 2014-08-18 01:00:00 to 2014-09-07 23:00:00
Persisted predictions detail of BSCTRFM_TPU_055_00_1637863002_MT_002_024_00
Loaded test time series for MT_003, which spans from 2014-08-18 01:00:00 to 2014-09-07 23:00:00
Persisted predictions detail of BSCTRFM_TPU_055_00_1637863002_MT_003_024_00
Loaded test time series for MT_004, which spans from 2014-08-18 01:00:00 to 2014-09-07 23:00:00
Persisted predictions detail of BSCTRFM_TPU_055_00_1637863002_MT_004_024_00
Loaded test time series for MT_005, which spans from 2014-08-18 01:00:00 to 2014-09-07 23:00:00
Persisted predictions detail of BSCTRFM_TPU_055_00_1637863002_MT_005_024_00
Loaded test time series for MT_006

Persisted predictions detail of BSCTRFM_TPU_055_00_1637863002_MT_048_024_00
Loaded test time series for MT_049, which spans from 2014-08-18 01:00:00 to 2014-09-07 23:00:00
Persisted predictions detail of BSCTRFM_TPU_055_00_1637863002_MT_049_024_00
Loaded test time series for MT_050, which spans from 2014-08-18 01:00:00 to 2014-09-07 23:00:00
Persisted predictions detail of BSCTRFM_TPU_055_00_1637863002_MT_050_024_00
Loaded test time series for MT_051, which spans from 2014-08-18 01:00:00 to 2014-09-07 23:00:00
Persisted predictions detail of BSCTRFM_TPU_055_00_1637863002_MT_051_024_00
Loaded test time series for MT_052, which spans from 2014-08-18 01:00:00 to 2014-09-07 23:00:00
Persisted predictions detail of BSCTRFM_TPU_055_00_1637863002_MT_052_024_00
Loaded test time series for MT_053, which spans from 2014-08-18 01:00:00 to 2014-09-07 23:00:00
Persisted predictions detail of BSCTRFM_TPU_055_00_1637863002_MT_053_024_00
Loaded test time series for MT_054, which spans from 2014-08-18 

Persisted predictions detail of BSCTRFM_TPU_055_00_1637863002_MT_096_024_00
Loaded test time series for MT_097, which spans from 2014-08-18 01:00:00 to 2014-09-07 23:00:00
Persisted predictions detail of BSCTRFM_TPU_055_00_1637863002_MT_097_024_00
Loaded test time series for MT_098, which spans from 2014-08-18 01:00:00 to 2014-09-07 23:00:00
Persisted predictions detail of BSCTRFM_TPU_055_00_1637863002_MT_098_024_00
Loaded test time series for MT_099, which spans from 2014-08-18 01:00:00 to 2014-09-07 23:00:00
Persisted predictions detail of BSCTRFM_TPU_055_00_1637863002_MT_099_024_00
Loaded test time series for MT_100, which spans from 2014-08-18 01:00:00 to 2014-09-07 23:00:00
Persisted predictions detail of BSCTRFM_TPU_055_00_1637863002_MT_100_024_00
Loaded test time series for MT_101, which spans from 2014-08-18 01:00:00 to 2014-09-07 23:00:00
Persisted predictions detail of BSCTRFM_TPU_055_00_1637863002_MT_101_024_00
Loaded test time series for MT_102, which spans from 2014-08-18 

Persisted predictions detail of BSCTRFM_TPU_055_00_1637863002_MT_144_024_00
Loaded test time series for MT_145, which spans from 2014-08-18 01:00:00 to 2014-09-07 23:00:00
Persisted predictions detail of BSCTRFM_TPU_055_00_1637863002_MT_145_024_00
Loaded test time series for MT_146, which spans from 2014-08-18 01:00:00 to 2014-09-07 23:00:00
Persisted predictions detail of BSCTRFM_TPU_055_00_1637863002_MT_146_024_00
Loaded test time series for MT_147, which spans from 2014-08-18 01:00:00 to 2014-09-07 23:00:00
Persisted predictions detail of BSCTRFM_TPU_055_00_1637863002_MT_147_024_00
Loaded test time series for MT_148, which spans from 2014-08-18 01:00:00 to 2014-09-07 23:00:00
Persisted predictions detail of BSCTRFM_TPU_055_00_1637863002_MT_148_024_00
Loaded test time series for MT_149, which spans from 2014-08-18 01:00:00 to 2014-09-07 23:00:00
Persisted predictions detail of BSCTRFM_TPU_055_00_1637863002_MT_149_024_00
Loaded test time series for MT_150, which spans from 2014-08-18 

Persisted predictions detail of BSCTRFM_TPU_055_00_1637863002_MT_192_024_00
Loaded test time series for MT_193, which spans from 2014-08-18 01:00:00 to 2014-09-07 23:00:00
Persisted predictions detail of BSCTRFM_TPU_055_00_1637863002_MT_193_024_00
Loaded test time series for MT_194, which spans from 2014-08-18 01:00:00 to 2014-09-07 23:00:00
Persisted predictions detail of BSCTRFM_TPU_055_00_1637863002_MT_194_024_00
Loaded test time series for MT_195, which spans from 2014-08-18 01:00:00 to 2014-09-07 23:00:00
Persisted predictions detail of BSCTRFM_TPU_055_00_1637863002_MT_195_024_00
Loaded test time series for MT_196, which spans from 2014-08-18 01:00:00 to 2014-09-07 23:00:00
Persisted predictions detail of BSCTRFM_TPU_055_00_1637863002_MT_196_024_00
Loaded test time series for MT_197, which spans from 2014-08-18 01:00:00 to 2014-09-07 23:00:00
Persisted predictions detail of BSCTRFM_TPU_055_00_1637863002_MT_197_024_00
Loaded test time series for MT_198, which spans from 2014-08-18 

...

/home/developer/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:207: RuntimeWarning: divide by zero encountered in double_scalars
/home/developer/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:212: RuntimeWarning: divide by zero encountered in double_scalars
/home/developer/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:207: RuntimeWarning: divide by zero encountered in double_scalars
/home/developer/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:212: RuntimeWarning: divide by zero encountered in double_scalars
/home/developer/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:207: RuntimeWarning: divide by zero encountered in double_scalars
/home/developer/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:212: RuntimeWarning: divide by zero encountered in double_scalars
/home/developer/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:207: RuntimeWarning: divide by zero encountered in double_scalars
/home/

Persisted predictions detail of BSCTRFM_TPU_056_01_1637868965_MT_223_024_00
Loaded test time series for MT_224, which spans from 2014-08-18 01:00:00 to 2014-09-07 23:00:00
Persisted predictions detail of BSCTRFM_TPU_056_01_1637868965_MT_224_024_00
Loaded test time series for MT_225, which spans from 2014-08-18 01:00:00 to 2014-09-07 23:00:00
Persisted predictions detail of BSCTRFM_TPU_056_01_1637868965_MT_225_024_00
Loaded test time series for MT_226, which spans from 2014-08-18 01:00:00 to 2014-09-07 23:00:00
Persisted predictions detail of BSCTRFM_TPU_056_01_1637868965_MT_226_024_00
Loaded test time series for MT_227, which spans from 2014-08-18 01:00:00 to 2014-09-07 23:00:00
Persisted predictions detail of BSCTRFM_TPU_056_01_1637868965_MT_227_024_00
Loaded test time series for MT_228, which spans from 2014-08-18 01:00:00 to 2014-09-07 23:00:00
Persisted predictions detail of BSCTRFM_TPU_056_01_1637868965_MT_228_024_00
Loaded test time series for MT_229, which spans from 2014-08-18 

Persisted predictions detail of BSCTRFM_TPU_056_01_1637868965_MT_271_024_00
Loaded test time series for MT_272, which spans from 2014-08-18 01:00:00 to 2014-09-07 23:00:00
Persisted predictions detail of BSCTRFM_TPU_056_01_1637868965_MT_272_024_00
Loaded test time series for MT_273, which spans from 2014-08-18 01:00:00 to 2014-09-07 23:00:00
Persisted predictions detail of BSCTRFM_TPU_056_01_1637868965_MT_273_024_00
Loaded test time series for MT_274, which spans from 2014-08-18 01:00:00 to 2014-09-07 23:00:00
Persisted predictions detail of BSCTRFM_TPU_056_01_1637868965_MT_274_024_00
Loaded test time series for MT_275, which spans from 2014-08-18 01:00:00 to 2014-09-07 23:00:00
Persisted predictions detail of BSCTRFM_TPU_056_01_1637868965_MT_275_024_00
Loaded test time series for MT_276, which spans from 2014-08-18 01:00:00 to 2014-09-07 23:00:00
Persisted predictions detail of BSCTRFM_TPU_056_01_1637868965_MT_276_024_00
Loaded test time series for MT_277, which spans from 2014-08-18 

Persisted predictions detail of BSCTRFM_TPU_056_01_1637868965_MT_319_024_00
Loaded test time series for MT_320, which spans from 2014-08-18 01:00:00 to 2014-09-07 23:00:00
Persisted predictions detail of BSCTRFM_TPU_056_01_1637868965_MT_320_024_00
Loaded test time series for MT_321, which spans from 2014-08-18 01:00:00 to 2014-09-07 23:00:00
Persisted predictions detail of BSCTRFM_TPU_056_01_1637868965_MT_321_024_00
Loaded test time series for MT_322, which spans from 2014-08-18 01:00:00 to 2014-09-07 23:00:00
Persisted predictions detail of BSCTRFM_TPU_056_01_1637868965_MT_322_024_00
Loaded test time series for MT_323, which spans from 2014-08-18 01:00:00 to 2014-09-07 23:00:00
Persisted predictions detail of BSCTRFM_TPU_056_01_1637868965_MT_323_024_00
Loaded test time series for MT_324, which spans from 2014-08-18 01:00:00 to 2014-09-07 23:00:00
Persisted predictions detail of BSCTRFM_TPU_056_01_1637868965_MT_324_024_00
Loaded test time series for MT_325, which spans from 2014-08-18 

Persisted predictions detail of BSCTRFM_TPU_056_01_1637868965_MT_367_024_00
Loaded test time series for MT_368, which spans from 2014-08-18 01:00:00 to 2014-09-07 23:00:00
Persisted predictions detail of BSCTRFM_TPU_056_01_1637868965_MT_368_024_00
Loaded test time series for MT_369, which spans from 2014-08-18 01:00:00 to 2014-09-07 23:00:00
Persisted predictions detail of BSCTRFM_TPU_056_01_1637868965_MT_369_024_00
Loaded test time series for MT_370, which spans from 2014-08-18 01:00:00 to 2014-09-07 23:00:00
Persisted predictions detail of BSCTRFM_TPU_056_01_1637868965_MT_370_024_00


In [58]:
# ND and NRMSE become inf(inite) for MT-223 because true values average = 0

predictions_detail['MT_223']

,model_id,execution,dataset,inference,customer_id,string_timestamps,predictions,targets,mae,nd,rmse,nrmse,smape
0,BSCTRFM_TPU_056,1,test,024,MT_223,"[2014-09-01 00:00:00, 2014-09-01 01:00:00, 201...","[0.0003578662872314453, 0.0003317892551422119,...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0.000295,inf,0.000429,inf,2.0
1,BSCTRFM_TPU_056,1,test,024,MT_223,"[2014-09-02 00:00:00, 2014-09-02 01:00:00, 201...","[0.0005418062210083008, 0.0003879964351654053,...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0.000311,inf,0.000373,inf,2.0
2,BSCTRFM_TPU_056,1,test,024,MT_223,"[2014-09-03 00:00:00, 2014-09-03 01:00:00, 201...","[0.0004285275936126709, 0.0007177889347076416,...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0.000293,inf,0.000375,inf,2.0
3,BSCTRFM_TPU_056,1,test,024,MT_223,"[2014-09-04 00:00:00, 2014-09-04 01:00:00, 201...","[0.00038877129554748535, 0.0002399384975433349...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0.000354,inf,0.000443,inf,2.0
4,BSCTRFM_TPU_056,1,test,024,MT_223,"[2014-09-05 00:00:00, 2014-09-05 01:00:00, 201...","[0.00018918514251708984, 0.0004921257495880127...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0.000383,inf,0.000474,inf,2.0
5,BSCTRFM_TPU_056,1,test,024,MT_223,"[2014-09-06 00:00:00, 2014-09-06 01:00:00, 201...","[0.0001347362995147705, 0.0010385513305664062,...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0.000272,inf,0.000362,inf,2.0
6,BSCTRFM_TPU_056,1,test,024,MT_223,"[2014-09-07 00:00:00, 2014-09-07 01:00:00, 201...","[0.0005478560924530029, 0.0003924369812011719,...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0.000293,inf,0.000377,inf,2.0
